In [7]:
%load_ext autoreload
%autoreload 2
import sys
s = "C:/users/nathan/documents/dropbox/python/jpnotebooks/Sessions/"
if s not in sys.path:
    sys.path.append(s)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import collections
import inspect
OD = collections.OrderedDict
from cfg_merge.sys_vars import _parse_sysvars
from cfg_merge.patcher.parse_config import parse_config

class NotFound(KeyError):
    """ Signal that variable was not found in the list.
    To be raised by user code as a signal to `compare_lists`. 
    """
    pass

NotFoundErrors = (NotFound, KeyError, IndexError)

def merge_lists(cf, of, nf, getvar, skipvar, vequal, notfound, setvar):
    for name in nf:
        if skipvar(name):
            continue
        nv = getvar(nf, name)
        try:
            cv = getvar(cf, name)
            ov = getvar(of, name)
        except NotFoundErrors:
            v = notfound(name)
            if v is None:
                v = nv
        else:
            
            if vequal(nv, ov) and vequal(ov, cv):
                v = nv
            elif not vequal(nv, ov) and vequal(ov, cv):
                v = nv                
            elif vequal(nv, ov) and not vequal(ov, cv):
                # XXX Conflict
                v = cv
            elif vequal(nv, cv) and not vequal(ov, cv):
                v = nv
            elif not vequal(nv, cv) and not vequal(cv, ov):
                # XXX Conflict
                v = cv
            else:
                # should be unreachable
                raise NameError("Unreachable code for %r"%name)
        setvar(name, v)
        
        
def merge_sysvars_inner(cf, of, nf, new, user, force, transnew, transold):
    
    # Callbacks
    def getvar(f,name):
        if f == nf:
            return f[name]
        else:
            if name in f:
                n = name
            else:
                n = transnew[name]
                if n not in f:
                    raise KeyError(n)
            return f[n]
        
    def skipvar(n):
        # handle force & such here
        if n in new:
            skip = True 
        elif n in user:
            nf[n].val = getvar(cf, n).val
            skip = True
        elif n in force:
            nf[n].val = force[n]
            skip = True
        else:
            skip = False
        return skip
        
    def vequal(a,b):
        return a.val == b.val
    def notfound(n):
        if n not in nf:
            print("Old Name: %s"%n)
        return None
    
    def setvar(name, v):
        #
        if nf[name].val != v.val:
            print("Updating %-45s: %.5f -> %.5f"%(name, nf[name].val, v.val))
            nf[name] = v
    
    return merge_lists(cf, of, nf, getvar, skipvar, vequal, notfound, setvar)
    



In [17]:
_FS = frozenset
def merge_sysvars(cff, off, nff, outf=None, new=None, user=None, force=None, tn=None, to=None):
    """
    1. Parse
    2. Flatten ("Group.Varname" = "Value")
    3. Merge
    4. Output
    """
    def parse_sysvars(ff): 
        return _parse_sysvars(ff)
    
    def _flatten(LV):
        d = OD()
        for g in LV.val:
            for v in g.val:
                d[g.name+"."+v.name] = v
        return d
            
    cLV = _parse_sysvars(cff)
    oLV = _parse_sysvars(off)
    nLV = _parse_sysvars(nff)
    
    cf  = _flatten(cLV)
    of  = _flatten(oLV)
    nf  = _flatten(nLV)
    
    # unpack args...
    new = new or _FS()
    user = user or _FS()
    force = force or {}
    tn = tn or {}
    to = to or {}
    
    merge_sysvars_inner(cf, of, nf, new, user, force, tn, to)

    if outf:
        with open(outf, 'w') as f:
            f.write(nLV.toxml())

In [19]:


# Example using PBS 3 CDI 3
cff = 'D:\\Customer_Backups\\000311T2902 180102\\Rio_Config\\System Variables.cfg'
off = 'D:\\auto_hd_install\\default configs\\IC3405 Rev C\\Mag 3 150727\\System Variables.cfg'
nff = 'D:\\auto_hd_install\\default configs\\IM00226 Rev B\\Mag 3\\System Variables.sys'

o = parse_config('test.sysvars.patch')
merge_sysvars(cff, off, nff, "test.xml", o.use_new, o.use_user, o.force, o.translate_new, o.translate_old)

Translating pH.Mismatch Thresh -> pH.Max Delta
Translating pH.Mismatch Thresh -> pH.Max Delta
Updating pH.CO2 Auto Max (%)                          : 100.00000 -> 7.50000
Updating pH.Base Manual Max (%)                       : 50.00000 -> 0.00000
Updating pH.Base Auto Max (%)                         : 50.00000 -> 0.00000
Updating pH.Deadband                                  : 0.02000 -> 0.01000
Updating DO.N2 I Time (min)                           : 50.00000 -> 1.00000
Updating Gas Data.Mismatch Thresh (V)                 : 0.10000 -> 0.20000
Updating Gas Data.Manual Max (LPM)                    : 0.50000 -> 1.00000
Updating Process Alarms.Agitation Low Low (RPM)       : 10.00000 -> 30.00000
Updating Process Alarms.Agitation Low (RPM)           : 15.00000 -> 32.00000
Updating Process Alarms.Agitation High (RPM)          : 35.00000 -> 38.00000
Updating Process Alarms.Agitation High High (RPM)     : 38.00000 -> 40.00000
Updating Process Alarms.Temp High (C)                 : 38.00000 -> 